In [ ]:
# main.py
# runs the simulation, and generates plots to visualize it

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import geopandas as gpd
from tqdm import trange, tqdm

from agent_torch.core import Registry, Runner
from agent_torch.core.helpers import read_config, read_from_file
from substeps import *
from helpers import *

In [ ]:
# get a geo plot of all the streets in boston
gdf = gpd.read_file('data/simulator/boston-streets.geojson')
gdf = gdf[["STREET_ID", "geometry"]]
gdf["street"] = gdf["STREET_ID"]
gdf = gdf.drop("STREET_ID", axis = 1)

In [ ]:
# setup the simulation based on the configuration
config = read_config("./config.yaml")
metadata = config.get("simulation_metadata")
num_episodes = metadata.get("num_episodes")
num_steps_per_episode = metadata.get("num_steps_per_episode")
visualize = metadata.get("visualize")

registry = Registry()
registry.register(read_from_file, "read_from_file", "initialization")

runner = Runner(config, registry)
runner.init()

In [ ]:
# run the simulation for the given number of steps

dfs = []
images = []
runner.reset()
for step in trange(num_steps_per_episode, desc=f":: running simulation"):
    runner.step(1)

    # store the data of solar cost incurred, so we can plot it
    # and compare it to grid costs later
    x = np.array(runner.state["agents"]["bap"]["position"]).flatten().astype(int)
    y = np.array(runner.state["agents"]["bap"]["wallet"]).flatten()

    # create a dataframe for easy plotting
    df = pd.DataFrame(np.array((x, y)).T, columns = ["street", "X"])
    dfs.append(df)

In [ ]:
# generate the plots for each iteration, and then compile those
# plots into a gif

for i, df in enumerate(tqdm(dfs, desc=f":: generating plots")):
    plot = gdf.plot(figsize = (16, 16), color = "gray")
    mdf = gdf.merge(df, on = "street")
    mdf.plot(column = "X", cmap="OrRd_r", figsize = (16, 16), ax = plot)

    plt.savefig(f"plots/step-{i}.jpg")
    images.append(f"plots/step-{i}.jpg")

frames = [imageio.imread(f"plots/step-{i}.jpg") for i in range(0, 20)]
imageio.mimsave(f"solar-usage.gif", frames, fps=2)